In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("contain_prev.csv", index_col=0) #이전에 던진 zone을 포함하는 데이터

In [3]:
data

,pitch_name,pitcher_x,description,stand,p_throws,plate_x,plate_z,inning,pitch_number,zone_group,prev_zone,y
0,5,676879,83,2,1,0.71,2.94,1,1,1,0.0,3.0
1,3,676879,66,2,1,-1.29,0.84,1,2,1,3.0,13.0
2,3,676879,83,2,1,-0.40,2.27,1,3,1,13.0,4.0
3,1,676879,83,2,1,1.15,3.25,1,4,1,4.0,12.0
4,3,676879,71,2,1,0.84,2.33,1,5,1,12.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
305863,1,669203,66,1,2,1.69,1.42,3,2,3,8.0,14.0
305864,4,669203,83,1,2,0.09,1.36,3,3,3,14.0,14.0
305865,5,669203,66,1,2,2.09,2.88,3,4,3,14.0,12.0
305866,2,669203,66,1,2,-0.70,0.84,3,5,3,12.0,13.0


In [6]:
X = data.drop("y", axis=1)

In [7]:
y = data["y"].astype(int)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, shuffle=True, test_size=0.3, random_state=42, stratify=x_train["zone_group"])

In [11]:
#jupyter에서는 paycaret 이 잘 돌아가지 않아 colab에서 실행 -> 가장 좋은 모델을 가져옴
#!pip install pycaret[full]
#!pip install catboost

In [15]:
#!pip install catboost

In [ ]:
#y값 바꿔주기 Xgboost 가 label을 0에서 순차적으로 처리

In [13]:
Y_train = Y_train-1

In [14]:
Y_train = Y_train.replace(14,9)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, shuffle=True, test_size=0.3, random_state=42, stratify=X_train["zone_group"])

In [18]:
from xgboost import XGBClassifier

# 객체 생성
model = XGBClassifier()
xgb_model = model.fit(x_train, y_train, early_stopping_rounds=100, 
                        eval_metric='mlogloss',eval_set=[(x_val, y_val)])

[0]	validation_0-mlogloss:1.15108


C:\Users\Jeong Hyunji\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\Jeong Hyunji\anaconda3\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-mlogloss:0.88346
[2]	validation_0-mlogloss:0.71274
[3]	validation_0-mlogloss:0.59533
[4]	validation_0-mlogloss:0.51161
[5]	validation_0-mlogloss:0.45062
[6]	validation_0-mlogloss:0.40561
[7]	validation_0-mlogloss:0.37192
[8]	validation_0-mlogloss:0.34645
[9]	validation_0-mlogloss:0.32708
[10]	validation_0-mlogloss:0.31232
[11]	validation_0-mlogloss:0.30080
[12]	validation_0-mlogloss:0.29177
[13]	validation_0-mlogloss:0.28464
[14]	validation_0-mlogloss:0.27896
[15]	validation_0-mlogloss:0.27436
[16]	validation_0-mlogloss:0.27076
[17]	validation_0-mlogloss:0.26786
[18]	validation_0-mlogloss:0.26538
[19]	validation_0-mlogloss:0.26341
[20]	validation_0-mlogloss:0.26188
[21]	validation_0-mlogloss:0.26037
[22]	validation_0-mlogloss:0.25925
[23]	validation_0-mlogloss:0.25819
[24]	validation_0-mlogloss:0.25726
[25]	validation_0-mlogloss:0.25659
[26]	validation_0-mlogloss:0.25595
[27]	validation_0-mlogloss:0.25548
[28]	validation_0-mlogloss:0.25510
[29]	validation_0-mlogloss:0.

하이퍼파라미터튜닝 역시 colab에서 진행

In [19]:
y_pred = xgb_model.predict(X_test)

In [20]:
y_pred = y_pred + 1

In [21]:
y_pred[y_pred == 10] = 15

In [22]:
pred = pd.DataFrame(y_pred, columns=["pred"])

In [24]:
original = Y_test.to_frame()

In [25]:
original = original.reset_index(drop=True)

In [26]:
original = original.astype(int)

In [27]:
result = pd.merge(original,pred, left_index=True, right_index=True)

In [28]:
result

,y,pred
0,12,12
1,2,2
2,9,9
3,14,14
4,13,13
...,...,...
91756,2,5
91757,2,2
91758,14,14
91759,9,9


In [29]:
from sklearn.metrics import accuracy_score

In [30]:
accuracy = accuracy_score(original, y_pred)
accuracy

0.8897352905918636